In [6]:
import numpy as np
import pandas as pd
import json
import networkx as nx
import itertools
import time

In [2]:
# fonction pour générer les résultats du secret santa
def secret_santa(liste_participants, config):
    '''
    la fonction prend en arguments une liste de participants, et une configuration qui définit:
    - les relations offrant > recevant prédéfinies
    - les relations offrant > recevant à exclure  
    elle renvoie une liste de couples (offrant, recevant)
    '''
    resultats = []
    offrants_restant = liste_participants.copy()
    recevants_restant = liste_participants.copy()
    exclusions = config['exclusions'].copy()
    obligations = config['obligations'].copy()

    # on traite d'abord les obligations
    print('traitement des obligations')
    for offrant, recevant in obligations:
        print('offrant: {}, recevant: {}'.format(offrant, recevant))
        resultats.append((offrant, recevant))
        offrants_restant.pop(offrants_restant.index(offrant))
        recevants_restant.pop(recevants_restant.index(recevant))
        
    # on parcours au hasard la liste des participants
    print('parcours de la liste')
    for i_offrant_a_traiter in np.random.permutation(len(offrants_restant)):
        offrant_a_traiter = offrants_restant[i_offrant_a_traiter]
        print('offrant: {}'.format(offrant_a_traiter))
        # on tire au hasard les recevants restants
        for i_recevant_possible in np.random.permutation(len(recevants_restant)):
            recevant_a_traiter = recevants_restant[i_recevant_possible]
            print('recevant: {}'.format(recevant_a_traiter))
            # vérification des exclusions
            if (offrant_a_traiter, recevant_a_traiter) not in exclusions and offrant_a_traiter != recevant_a_traiter:
                resultats.append((offrant_a_traiter, recevant_a_traiter))
                recevants_restant.pop(i_recevant_possible)
                break
            else:
                continue
    return(resultats)

In [3]:
liste_participants = ['Papa', 'Maman', 'Didier', 'Anne', 'Cécile', 'Mamy', 'Bruno']
config = {
    'exclusions':[
        ('Papa', 'Maman'),
        ('Maman', 'Papa'),
        ('Cecile', 'Bruno'),
        ('Bruno', 'Cecile'),
        ('Anne', 'Didier'),
        ('Didier', 'Anne')],
    'obligations':[
        ('Bruno', 'Papa'),
        ('Didier', 'Maman')]
}
secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

traitement des obligations
offrant: Bruno, recevant: Papa
offrant: Didier, recevant: Maman
parcours de la liste
offrant: Anne
recevant: Anne
recevant: Mamy
offrant: Mamy
recevant: Anne
offrant: Cécile
recevant: Bruno
offrant: Papa
recevant: Didier
offrant: Maman
recevant: Cécile


,offrant,recevant
0,Bruno,Papa
1,Didier,Maman
2,Anne,Mamy
3,Mamy,Anne
4,Cécile,Bruno
5,Papa,Didier
6,Maman,Cécile


In [4]:
liste_participants = ['TERRIER Viktor', 'FINET Sebastien', 'GIROD Marie', 'COGEN Florent', 'DUMAS Jonathan', 'LITTLE Emily', 'DUSSARTRE Virginie', 'HAMDANE Ibtissam', 
                      'ARULPRAGASAM Steven', 'MOTHE Axel', 'BUSTARRET Quentin', 'LEMETAYER Bruno']
config = {
    'exclusions':[],
    'obligations':[]}
secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

traitement des obligations
parcours de la liste
offrant: HAMDANE Ibtissam
recevant: ARULPRAGASAM Steven
offrant: GIROD Marie
recevant: LEMETAYER Bruno
offrant: BUSTARRET Quentin
recevant: MOTHE Axel
offrant: ARULPRAGASAM Steven
recevant: GIROD Marie
offrant: DUSSARTRE Virginie
recevant: DUMAS Jonathan
offrant: DUMAS Jonathan
recevant: LITTLE Emily
offrant: TERRIER Viktor
recevant: BUSTARRET Quentin
offrant: LITTLE Emily
recevant: FINET Sebastien
offrant: LEMETAYER Bruno
recevant: DUSSARTRE Virginie
offrant: COGEN Florent
recevant: TERRIER Viktor
offrant: FINET Sebastien
recevant: HAMDANE Ibtissam
offrant: MOTHE Axel
recevant: COGEN Florent


,offrant,recevant
0,HAMDANE Ibtissam,ARULPRAGASAM Steven
1,GIROD Marie,LEMETAYER Bruno
2,BUSTARRET Quentin,MOTHE Axel
3,ARULPRAGASAM Steven,GIROD Marie
4,DUSSARTRE Virginie,DUMAS Jonathan
5,DUMAS Jonathan,LITTLE Emily
6,TERRIER Viktor,BUSTARRET Quentin
7,LITTLE Emily,FINET Sebastien
8,LEMETAYER Bruno,DUSSARTRE Virginie
9,COGEN Florent,TERRIER Viktor


In [5]:
with open('data/config_famille.json') as f:
    config = json.load(f)
liste_participants = pd.read_csv('data/participants_famille.csv', header=None, names=['participants'])['participants'].to_list()

secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

traitement des obligations
offrant: Bruno, recevant: Papa
offrant: Didier, recevant: Maman
parcours de la liste
offrant: Papa
recevant: Bruno
offrant: Mamy
recevant: Didier
offrant: Anne
recevant: Anne
recevant: Cécile
offrant: Maman
recevant: Anne
offrant: Cécile
recevant: Mamy


,offrant,recevant
0,Bruno,Papa
1,Didier,Maman
2,Papa,Bruno
3,Mamy,Didier
4,Anne,Cécile
5,Maman,Anne
6,Cécile,Mamy


In [6]:
with open('config_exemple.json') as f:
    config = json.load(f)
liste_participants = pd.read_csv('participants_exemple.csv', header=None, names=['participants'])['participants'].to_list()

secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

traitement des obligations
offrant: Alpha, recevant: Beta
parcours de la liste
offrant: Gamma
recevant: Alpha
offrant: Delta
recevant: Delta
recevant: Gamma
offrant: Omega
recevant: Omega
recevant: Delta
offrant: Beta
recevant: Omega


,offrant,recevant
0,Alpha,Beta
1,Gamma,Alpha
2,Delta,Gamma
3,Omega,Delta
4,Beta,Omega


# Avec des graphes

In [12]:
# fonction pour générer les résultats du secret santa
def secret_santa(liste_participants, config, timeout_min = 1):
    '''
    la fonction prend en arguments une liste de participants, et une configuration qui définit:
    - les relations offrant > recevant prédéfinies
    - les relations offrant > recevant à exclure  
    elle renvoie une liste de couples (offrant, recevant)
    '''
    # création du graphe avec les participants
    exclusions = config['exclusions'].copy()
    edge_list = pd.DataFrame(
        [[item[0], item[1], 0] for item in itertools.product(liste_participants, liste_participants) if item[0] != item[1] and [item[0], item[1]] not in exclusions], 
        columns = ['source', 'target', 'retenu'])
    G = nx.from_pandas_edgelist(
        edge_list, 
        edge_attr = 'retenu',
        create_using = nx.DiGraph())
    
    # on traite d'abord les obligations
    obligations = config['obligations'].copy()
    offrants_obliges = []
    recevants_obliges = []
    for offrant, recevant in obligations:
        nx.set_edge_attributes(G, {(offrant, recevant): {'retenu': 1}})
        offrants_obliges.append(offrant)
        recevants_obliges.append(recevant)
       
    # on parcours au hasard la liste des participants
    cadeaux_a_faire = len(liste_participants) - len(obligations)
    parcours_offrant = []
    parcours_recevant = []
    # initialisation du timeout
    timeout = time.time() + 60*timeout_min
    # initialisation du message
    message = ""
    
    while len(parcours_offrant) < cadeaux_a_faire and len(parcours_recevant) < cadeaux_a_faire:
        # on arrête la boucle si le temps dépasse la limite
        if time.time() > timeout:
            break
        offrants_possibles = [offrant for offrant in liste_participants if (offrant not in parcours_offrant and offrant not in offrants_obliges)]
        offrant_a_traiter = offrants_possibles[np.random.randint(len(offrants_possibles))]
        recevants_possibles = [recevant for recevant in list(G[offrant_a_traiter]) if (recevant not in parcours_recevant and recevant not in recevants_obliges)]
        if len(recevants_possibles) > 0:
            # on tire au hasard les recevants restants parmi les target 
            recevant_a_traiter = recevants_possibles[np.random.randint(len(recevants_possibles))]
            nx.set_edge_attributes(G, {(offrant_a_traiter, recevant_a_traiter): {'retenu': 1}})
            parcours_offrant.append(offrant_a_traiter)
            parcours_recevant.append(recevant_a_traiter)
        else:
            # retour arrière
            nx.set_edge_attributes(G, {(parcours_offrant.pop(), parcours_recevant.pop()): {'retenu': 0}})
    
    if len(parcours_offrant) == cadeaux_a_faire and len(parcours_recevant) == cadeaux_a_faire:
        message = "solution trouvée"
    else:
        message = 'pas de solution trouvée'
    return(message, G, [(offrant, recevant) for offrant, recevant, edge in G.edges(data=True) if edge['retenu'] == 1])

In [14]:
message, G, secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

,offrant,recevant
0,Papa,Cécile
1,Maman,Didier
2,Didier,Maman
3,Anne,Mamy
4,Cécile,Bruno
5,Mamy,Anne
6,Bruno,Papa
